In [ ]:
import os

import pandas
import pandas as pd

In [ ]:
data_location = '..\data\endsong_'

# read first file to get df format
index = 0
df = pd.read_json(f'{data_location}{index}.json')
index += 1

# append the rest of the files
while os.path.exists(f'{data_location}{index}.json'):
    df = pd.concat([df, pd.read_json(f'{data_location}{index}.json')])
    index += 1

In [ ]:
df

separo los nombres

In [ ]:
metadata = df[['spotify_track_uri', 'master_metadata_album_artist_name', 'master_metadata_track_name', 'master_metadata_album_album_name']].drop_duplicates()

# stats por cancion

In [ ]:
groupby = 'master_metadata_album_album_name'

In [ ]:
groupby_uri = df.groupby(by=groupby)

In [ ]:
tracks_by_ms_played = groupby_uri[[groupby, 'ms_played']].count().sort_values(by='ms_played', ascending=False)[['ms_played']]
tracks_by_ms_played

In [ ]:
pd.merge(tracks_by_ms_played, metadata, on=[groupby], how='left')